In [1]:
# load the library
from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd
import re
import time
# indeed.com url
base_url = 'https://www.indeed.com/jobs?q=all+jobs+hiring&jt={}&l=Anywhere&sort={}&start={}'
sort_by = 'date'          # sort by data
#start_from = '&start='    # start page number
#pd.set_option('max_colwidth',500)    # to remove column limit (Otherwise, we'll lose some info)
df = pd.DataFrame()   # create a new data frame

In [2]:
start_time = time.time()
jobtype= ['fulltime','parttime']
i = 0
for job in jobtype:
    for page in range(1,5002): # page from 1 to 100 (last page we can scrape is 100)
        page = (page-1) * 10  
        url = base_url.format(job,sort_by,page)
        html = requests.get(url)
        time.sleep(1)
        soup = Soup(html.content, 'lxml')
        soupelements = soup.find_all(class_= "result" )
        for elem in soupelements: 
            #get job title
            try:
                job_title = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
            except:
                job_title=None
            #get company name
            try:
                comp_name = elem.find(class_='company').text.replace('\n', '')
            except:
                comp_name=None
                
            #get date of post
            try:
                job_posted = elem.find('span', attrs={'class': 'date'}).getText()
            except: 
                    None
            #get salary        
            try:
                salary = elem.find('span', {'class':'no-wrap'}).text
            except:
                salary= None
            #determine if job is fulltime or parttime
            job_type = job  
            #get url and href     
            home_url = "http://www.indeed.com"
            job_link = "%s%s" % (home_url,elem.find('a').get('href'))

            try:
                synopsis= elem.find('span', {'class':'summary'}).text.replace('\n', '')
            except: 
                synopsis = None
            #compile all into d
            df = df.append({'job_title': job_title,'company_name': comp_name,
                            'date_posted': job_posted,"salary": salary,'job_type':job_type,
                            "synopsis":synopsis, 'job_link': job_link}, ignore_index=True)
            i+= 1
            if i % 1000 == 0:  
                print(len(df),'You have ' + str(i) + ' results. ' + str(df.dropna().drop_duplicates().shape[0]) + " of these are good.")
            if len(df) >= 100000:
                break
end_time = time.time()
print("DONE! It took :", (end_time- start_time)/60,"mins", "to finish")                    

1000 You have 1000 results. 427 of these are good.
2000 You have 2000 results. 774 of these are good.
3000 You have 3000 results. 1050 of these are good.
4000 You have 4000 results. 1297 of these are good.
5000 You have 5000 results. 1521 of these are good.
6000 You have 6000 results. 1764 of these are good.
7000 You have 7000 results. 1998 of these are good.
8000 You have 8000 results. 2241 of these are good.
9000 You have 9000 results. 2495 of these are good.
10000 You have 10000 results. 2725 of these are good.
11000 You have 11000 results. 2974 of these are good.
12000 You have 12000 results. 3241 of these are good.
13000 You have 13000 results. 3509 of these are good.
14000 You have 14000 results. 3783 of these are good.
15000 You have 15000 results. 4103 of these are good.
16000 You have 16000 results. 4457 of these are good.
17000 You have 17000 results. 4827 of these are good.
18000 You have 18000 results. 5224 of these are good.
19000 You have 19000 results. 5609 of these are 

ConnectionError: HTTPSConnectionPool(host='www.indeed.com', port=443): Max retries exceeded with url: /jobs?q=all+jobs+hiring&jt=parttime&l=Anywhere&sort=date&start=31910 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x126dc9588>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [14]:
df.to_csv("df_indeed_final.csv")

In [16]:
for page in range(1,5002): # page from 1 to 100 (last page we can scrape is 100)
        page = (page-1) * 10  
        